In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

model_path = 'models/models_v3/t5_time_extractor_r_p_1000_1000'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

In [2]:
import json
import ast

def trans_list(list_str):
    list_with_json_strings = ast.literal_eval(list_str)

    # 第二步：遍历列表，将每个JSON字符串转换为字典
    result_list = []
    for json_str in list_with_json_strings:
        # 解析JSON字符串为字典
        data_dict = json.loads(json_str)
        result_list.append(data_dict)
    return result_list

def delete_space(text):
    return json.dumps(text, separators=(',', ':')).replace(" ", "").replace("\n", "").replace("\r", "").replace("\\n","")

def compare(predict_text, target_text):
    predict_list = trans_list(predict_text)
    target_list = trans_list(target_text)
    if len(predict_list) != len(target_list):
        return False
    for i in range(len(predict_list)):
        if delete_space(predict_list[i]) != delete_space(target_list[i]):
            return False
    return True

In [18]:
import pandas as pd
import json

predict_file_path = 'data/v3/predict/test_1000_r_p_fix.json'

dataset = json.load(open(predict_file_path))
error_list = []
for i in range(len(dataset)):
    if not compare(dataset[i]['predict_text'], dataset[i]['target_text']):
        error_list.append({
            "index":i,
            "input_text":dataset[i]['input_text'],
            "target_text":dataset[i]['target_text'],
            "predict_list":dataset[i]['predict_text'],
        })

In [19]:
len(error_list)

1

In [21]:
error_list

[{'index': 534,
  'input_text': '"学生日常课程安排里，对应上周二那节课是正常去上课的。"',
  'target_text': '[\'{"month":"none","week":"-1","weekday":"2","day":"none","term":"none"}\']',
  'predict_list': '[\'{"month":"none","week":"none","weekday":"none","day":"-2","term":"none"}\']'}]

In [13]:
import pandas as pd

file_path = 'data/v3/test_1000_r.csv'
index = 912

dataset = pd.read_csv(file_path)
input_texts = [e for e in dataset['input_text']]
target_texts = [e for e in dataset['target_text']]

input_texts[index], target_texts[index]

('下学期的最后一周请假',
 '[\'{"month": "none", "week": "-1", "weekday": "none", "day": "none", "term": "1"}\']')

In [14]:
import pandas as pd

file_path = 'data/v3/test_1000_r_p_fix.csv'
index = 912

dataset = pd.read_csv(file_path)
input_texts = [e for e in dataset['input_text']]
target_texts = [e for e in dataset['target_text']]

input_texts[index], target_texts[index]

('"因个人特殊情况，需在本学期最后一周请假，望批准。"',
 '[\'{"month":"none","week":"-1","weekday":"none","day":"none","term":"1"}\']')